# Reproduction des figures — Transferring Learning Trajectories of Neural Networks

**Article** : Chijiwa et al. — arXiv:2305.14122v2  
**Repo officiel** : `learning-transfer/`

Ce notebook orchestre la reproduction des figures de l'article en utilisant le dépôt officiel de Daiki.

## Pré-requis
- GPU CUDA disponible
- `torch`, `torchvision`, `pyyaml`, `pandas`, `matplotlib`, `scipy` installés
- Être dans le répertoire `learning-transfer/`

In [1]:
import os
import sys
import json
import subprocess
import time
from pathlib import Path

# S'assurer qu'on est dans le bon répertoire
REPO_ROOT = Path('/workspace') 
os.chdir(REPO_ROOT)
sys.path.insert(0, str(REPO_ROOT))

# Créer les répertoires nécessaires
for d in ['__outputs__', '__data__', '__sync__']:
    os.makedirs(d, exist_ok=True)

print(f'Working directory: {os.getcwd()}')
print(f'GPU available: {subprocess.run([sys.executable, "-c", "import torch; print(torch.cuda.is_available())"], capture_output=True, text=True).stdout.strip()}')

Working directory: /workspace
GPU available: True


In [2]:
# Vérifier que le stub EnsembleEvaluater existe
stub_path = REPO_ROOT / 'models' / 'ensemble_evaluater.py'
if not stub_path.exists():
    stub_path.write_text('class EnsembleEvaluater:\n    pass\n')
    print('Created EnsembleEvaluater stub')
else:
    print('EnsembleEvaluater stub already exists')

# Vérifier les imports critiques
import torch
import torchvision
import yaml
import scipy
import matplotlib
print(f'PyTorch: {torch.__version__}')
print(f'TorchVision: {torchvision.__version__}')
print(f'CUDA: {torch.cuda.is_available()} ({torch.cuda.get_device_name(0) if torch.cuda.is_available() else "N/A"})')

EnsembleEvaluater stub already exists
PyTorch: 1.12.1+cu113
TorchVision: 0.13.1+cu113
CUDA: True (NVIDIA GeForce RTX 4070 Laptop GPU)


## Utilitaires

Fonctions helper pour lancer les commandes et suivre la progression.

In [3]:
def run_cmd(command, exp_name, gpu_id=0, config='config.yaml', timeout=None):
    """Lance une commande exec_parallel.py et affiche la sortie en temps réel."""
    cmd = [
        sys.executable, 'exec_parallel.py', command, exp_name,
        '--gpu_id', str(gpu_id),
        '--config', config
    ]
    print(f'\n{"="*60}')
    print(f'Running: {" ".join(cmd)}')
    print(f'{"="*60}')
    start = time.time()
    
    result = subprocess.run(
        cmd,
        cwd=str(REPO_ROOT),
        capture_output=True,
        text=True,
        timeout=timeout
    )
    
    elapsed = time.time() - start
    print(f'\nCompleted in {elapsed:.1f}s (exit code: {result.returncode})')
    
    if result.stdout:
        # Afficher les dernières lignes pertinentes
        lines = result.stdout.strip().split('\n')
        display_lines = lines[-30:] if len(lines) > 30 else lines
        for line in display_lines:
            print(f'  {line}')
    
    if result.returncode != 0 and result.stderr:
        print(f'\nSTDERR (last 20 lines):')
        err_lines = result.stderr.strip().split('\n')[-20:]
        for line in err_lines:
            print(f'  {line}')
    
    return result


def check_outputs(exp_name, expected_count=None):
    """Vérifie les fichiers de sortie pour une expérience."""
    output_dir = REPO_ROOT / '__outputs__' / exp_name
    if not output_dir.exists():
        print(f'  No output directory for {exp_name}')
        return []
    
    files = sorted(output_dir.iterdir())
    json_files = [f for f in files if f.suffix == '.json' and 'transfer_results' in f.name]
    pth_files = [f for f in files if f.suffix == '.pth']
    
    print(f'  {exp_name}: {len(pth_files)} checkpoints, {len(json_files)} transfer_results')
    return json_files


def run_train_until_complete(exp_name, gpu_id=0, max_retries=10):
    """Lance la commande train en boucle jusqu'à ce que tous les seeds soient complètes."""
    for i in range(max_retries):
        print(f'\n--- Train attempt {i+1}/{max_retries} for {exp_name} ---')
        result = run_cmd('train', exp_name, gpu_id=gpu_id)
        if 'all patterns are executed' in (result.stdout or ''):
            print(f'All training seeds complete for {exp_name}')
            return True
    print(f'May need more retries for {exp_name}')
    return False

---
## Figure 5a — MNIST / MLP (Random Init → MNIST)

C'est l'expérience la plus simple et la plus rapide (~30 min).

### Étape 1 : Entraîner les modèles source/target

In [4]:
# Entraîner 6 seeds : [101, 102, 201, 202, 301, 302]
# La commande itère aléatoirement sur la grille hparams_grid.
# On la relance jusqu'à avoir couvert toutes les combinaisons.
run_train_until_complete('mnist_mlp_sgd', gpu_id=0)


--- Train attempt 1/10 for mnist_mlp_sgd ---

Running: /usr/bin/python3 exec_parallel.py train mnist_mlp_sgd --gpu_id 0 --config config.yaml

Completed in 204.3s (exit code: 0)
  Train Accuracy: 91.58888888888889
  Val Accuracy: 90.95
  [ 2026-02-11 15:29:13.282497 ] Epoch:  3
  Train Accuracy: 91.75555555555556
  Val Accuracy: 91.2
  [ 2026-02-11 15:29:15.751270 ] Epoch:  4
  Train Accuracy: 92.10000000000001
  Val Accuracy: 91.23333333333333
  [ 2026-02-11 15:29:18.218393 ] Epoch:  5
  Train Accuracy: 92.08333333333333
  Val Accuracy: 91.4
  [ 2026-02-11 15:29:20.759098 ] Epoch:  6
  Train Accuracy: 92.17222222222222
  Val Accuracy: 91.03333333333333
  [ 2026-02-11 15:29:23.275357 ] Epoch:  7
  Train Accuracy: 92.1648148148148
  Val Accuracy: 91.11666666666667
  [ 2026-02-11 15:29:25.773527 ] Epoch:  8
  Train Accuracy: 92.21851851851852
  Val Accuracy: 90.9
  [ 2026-02-11 15:29:28.217731 ] Epoch:  9
  Train Accuracy: 92.17407407407407
  Val Accuracy: 91.03333333333333
  Number of a

True

### Étape 2 : Transférer les trajectoires

4 méthodes : **GMT**, **FGMT**, **Naive** (noperm), **Oracle** (usetarget)

Chaque commande transfer produit :
- `transfer_results.json` — accuracies pendant le transfert (pour Fig 5a)
- `ft_results.json` — accuracies pendant le fine-tuning (pour Fig 6a)

In [6]:
# GMT Transfer
run_cmd('transfer', 'transfer_init-mnist_mlp', gpu_id=0)


Running: /usr/bin/python3 exec_parallel.py transfer transfer_init-mnist_mlp --gpu_id 0 --config config.yaml

Completed in 413.5s (exit code: 0)
  [Learning Transfer:6/10] Val Accuracy: 75.08333333333333
                        Matching Loss: tensor(42.1973, device='cuda:0')
  [Learning Transfer:7/10] Val Accuracy: 85.16666666666667
                        Matching Loss: tensor(55.4332, device='cuda:0')
  [Learning Transfer:8/10] Val Accuracy: 83.15
                        Matching Loss: tensor(44.9009, device='cuda:0')
  [Learning Transfer:9/10] Val Accuracy: 84.71666666666667
                        Matching Loss: tensor(48.1556, device='cuda:0')
  [Learning Transfer:10/10] Val Accuracy: 82.25
                        Matching Loss: tensor(54.9484, device='cuda:0')
  [Learning Transfer] Final Val Accuracy: 82.25
  val_accs: [62.983333333333334, 70.68333333333334, 80.30000000000001, 75.38333333333334, 74.28333333333333, 75.08333333333333, 85.16666666666667, 83.15, 84.71666666666667, 82

CompletedProcess(args=['/usr/bin/python3', 'exec_parallel.py', 'transfer', 'transfer_init-mnist_mlp', '--gpu_id', '0', '--config', 'config.yaml'], returncode=0, stdout="[DEBUG]\n0.0\n[DEBUG]\n398.62231254577637\n[DEBUG]\n[ExecParallel] Start job: epoch_ft_3--lr_ft_0.01--gm_iters_1--checkpoint_epochs_[-1, 9]--num_splits_10--split_scheduling_fn_1+math.cos((t/T)*math.pi)--fast_perm_optim_False--source_target_seeds_[201, 202]--\nPrefix: 2aaa3dcf2ab4ecb2e8df8e22ff53993a\n[Sync] start: job_id= 1770824200.2163525\n\nckp_epochs: [-1, 9]\n\n[Learning Transfer:1/10] Val Accuracy: 57.56666666666666\n                      Matching Loss: tensor(11.6997, device='cuda:0')\n[Learning Transfer:2/10] Val Accuracy: 69.56666666666666\n                      Matching Loss: tensor(20.1681, device='cuda:0')\n[Learning Transfer:3/10] Val Accuracy: 71.85000000000001\n                      Matching Loss: tensor(23.2758, device='cuda:0')\n[Learning Transfer:4/10] Val Accuracy: 67.08333333333333\n                 

In [7]:
# FGMT Transfer (version rapide)
run_cmd('transfer', 'transfer_init-mnist_mlp_fast', gpu_id=0)


Running: /home/alan/miniconda3/envs/transferring-learning-trajectories/bin/python exec_parallel.py transfer transfer_init-mnist_mlp_fast --gpu_id 0 --config config.yaml

Completed in 2.8s (exit code: 1)

STDERR (last 20 lines):
  Traceback (most recent call last):
    File "/home/alan/Transferring_Learning_Trajectories_of_NN/learning-transfer/exec_parallel.py", line 16, in <module>
      import commands
    File "/home/alan/Transferring_Learning_Trajectories_of_NN/learning-transfer/commands/__init__.py", line 7, in <module>
      importlib.import_module('.' + module[:-3], package='commands')
    File "/home/alan/miniconda3/envs/transferring-learning-trajectories/lib/python3.11/importlib/__init__.py", line 126, in import_module
      return _bootstrap._gcd_import(name[level:], package, level)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "/home/alan/Transferring_Learning_Trajectories_of_NN/learning-transfer/commands/train.py", line 10, in <module>
      fro

CompletedProcess(args=['/home/alan/miniconda3/envs/transferring-learning-trajectories/bin/python', 'exec_parallel.py', 'transfer', 'transfer_init-mnist_mlp_fast', '--gpu_id', '0', '--config', 'config.yaml'], returncode=1, stdout='', stderr='Traceback (most recent call last):\n  File "/home/alan/Transferring_Learning_Trajectories_of_NN/learning-transfer/exec_parallel.py", line 16, in <module>\n    import commands\n  File "/home/alan/Transferring_Learning_Trajectories_of_NN/learning-transfer/commands/__init__.py", line 7, in <module>\n    importlib.import_module(\'.\' + module[:-3], package=\'commands\')\n  File "/home/alan/miniconda3/envs/transferring-learning-trajectories/lib/python3.11/importlib/__init__.py", line 126, in import_module\n    return _bootstrap._gcd_import(name[level:], package, level)\n           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/home/alan/Transferring_Learning_Trajectories_of_NN/learning-transfer/commands/train.py", line 10, in <module>\n  

In [8]:
# Naive Transfer (pas de permutation)
run_cmd('transfer', 'transfer_init-mnist_mlp_noperm', gpu_id=0)


Running: /home/alan/miniconda3/envs/transferring-learning-trajectories/bin/python exec_parallel.py transfer transfer_init-mnist_mlp_noperm --gpu_id 0 --config config.yaml

Completed in 2.8s (exit code: 1)

STDERR (last 20 lines):
  Traceback (most recent call last):
    File "/home/alan/Transferring_Learning_Trajectories_of_NN/learning-transfer/exec_parallel.py", line 16, in <module>
      import commands
    File "/home/alan/Transferring_Learning_Trajectories_of_NN/learning-transfer/commands/__init__.py", line 7, in <module>
      importlib.import_module('.' + module[:-3], package='commands')
    File "/home/alan/miniconda3/envs/transferring-learning-trajectories/lib/python3.11/importlib/__init__.py", line 126, in import_module
      return _bootstrap._gcd_import(name[level:], package, level)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "/home/alan/Transferring_Learning_Trajectories_of_NN/learning-transfer/commands/train.py", line 10, in <module>
      f

CompletedProcess(args=['/home/alan/miniconda3/envs/transferring-learning-trajectories/bin/python', 'exec_parallel.py', 'transfer', 'transfer_init-mnist_mlp_noperm', '--gpu_id', '0', '--config', 'config.yaml'], returncode=1, stdout='', stderr='Traceback (most recent call last):\n  File "/home/alan/Transferring_Learning_Trajectories_of_NN/learning-transfer/exec_parallel.py", line 16, in <module>\n    import commands\n  File "/home/alan/Transferring_Learning_Trajectories_of_NN/learning-transfer/commands/__init__.py", line 7, in <module>\n    importlib.import_module(\'.\' + module[:-3], package=\'commands\')\n  File "/home/alan/miniconda3/envs/transferring-learning-trajectories/lib/python3.11/importlib/__init__.py", line 126, in import_module\n    return _bootstrap._gcd_import(name[level:], package, level)\n           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/home/alan/Transferring_Learning_Trajectories_of_NN/learning-transfer/commands/train.py", line 10, in <module>\n

In [9]:
# Oracle Transfer (utilise la vraie trajectoire cible)
run_cmd('transfer', 'transfer_init-mnist_mlp_usetarget', gpu_id=0)


Running: /home/alan/miniconda3/envs/transferring-learning-trajectories/bin/python exec_parallel.py transfer transfer_init-mnist_mlp_usetarget --gpu_id 0 --config config.yaml

Completed in 2.7s (exit code: 1)

STDERR (last 20 lines):
  Traceback (most recent call last):
    File "/home/alan/Transferring_Learning_Trajectories_of_NN/learning-transfer/exec_parallel.py", line 16, in <module>
      import commands
    File "/home/alan/Transferring_Learning_Trajectories_of_NN/learning-transfer/commands/__init__.py", line 7, in <module>
      importlib.import_module('.' + module[:-3], package='commands')
    File "/home/alan/miniconda3/envs/transferring-learning-trajectories/lib/python3.11/importlib/__init__.py", line 126, in import_module
      return _bootstrap._gcd_import(name[level:], package, level)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "/home/alan/Transferring_Learning_Trajectories_of_NN/learning-transfer/commands/train.py", line 10, in <module>
    

CompletedProcess(args=['/home/alan/miniconda3/envs/transferring-learning-trajectories/bin/python', 'exec_parallel.py', 'transfer', 'transfer_init-mnist_mlp_usetarget', '--gpu_id', '0', '--config', 'config.yaml'], returncode=1, stdout='', stderr='Traceback (most recent call last):\n  File "/home/alan/Transferring_Learning_Trajectories_of_NN/learning-transfer/exec_parallel.py", line 16, in <module>\n    import commands\n  File "/home/alan/Transferring_Learning_Trajectories_of_NN/learning-transfer/commands/__init__.py", line 7, in <module>\n    importlib.import_module(\'.\' + module[:-3], package=\'commands\')\n  File "/home/alan/miniconda3/envs/transferring-learning-trajectories/lib/python3.11/importlib/__init__.py", line 126, in import_module\n    return _bootstrap._gcd_import(name[level:], package, level)\n           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/home/alan/Transferring_Learning_Trajectories_of_NN/learning-transfer/commands/train.py", line 10, in <module

In [10]:
# Vérifier les résultats produits
for exp in ['transfer_init-mnist_mlp', 'transfer_init-mnist_mlp_fast',
            'transfer_init-mnist_mlp_noperm', 'transfer_init-mnist_mlp_usetarget']:
    check_outputs(exp)

  No output directory for transfer_init-mnist_mlp
  No output directory for transfer_init-mnist_mlp_fast
  No output directory for transfer_init-mnist_mlp_noperm
  No output directory for transfer_init-mnist_mlp_usetarget


### Étape 3 : Générer la figure PDF

In [ ]:
result = run_cmd('plot', 'section_4_1_mnist_mlp', gpu_id=0)

# Trouver et afficher le PDF généré
import glob
pdfs = glob.glob(str(REPO_ROOT / '__outputs__' / '**' / '*section_4-1*mnist*.pdf'), recursive=True)
if pdfs:
    print(f'\nFigure générée : {pdfs[0]}')
else:
    print('\nPDF non trouvé — vérifier les logs ci-dessus')


Running: /home/alan/miniconda3/envs/transferring-learning-trajectories/bin/python exec_parallel.py plot section_4_1_mnist_mlp --gpu_id 0 --config config.yaml

Completed in 2.8s (exit code: 1)

STDERR (last 20 lines):
  Traceback (most recent call last):
    File "/home/alan/Transferring_Learning_Trajectories_of_NN/learning-transfer/exec_parallel.py", line 16, in <module>
      import commands
    File "/home/alan/Transferring_Learning_Trajectories_of_NN/learning-transfer/commands/__init__.py", line 7, in <module>
      importlib.import_module('.' + module[:-3], package='commands')
    File "/home/alan/miniconda3/envs/transferring-learning-trajectories/lib/python3.11/importlib/__init__.py", line 126, in import_module
      return _bootstrap._gcd_import(name[level:], package, level)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "/home/alan/Transferring_Learning_Trajectories_of_NN/learning-transfer/commands/train.py", line 10, in <module>
      from commands.

### Afficher la figure générée

In [ ]:
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Chercher le PDF ou le convertir en PNG pour l'affichage
pdfs = glob.glob(str(REPO_ROOT / '__outputs__' / '**' / '*section_4-1*mnist*.pdf'), recursive=True)
if pdfs:
    # Convertir le PDF en PNG pour l'affichage dans le notebook
    pdf_path = pdfs[0]
    png_path = pdf_path.replace('.pdf', '.png')
    try:
        subprocess.run(['pdftoppm', '-png', '-r', '150', '-singlefile', pdf_path, png_path.replace('.png', '')],
                      check=True, capture_output=True)
        display(Image(filename=png_path, width=500))
    except (subprocess.CalledProcessError, FileNotFoundError):
        print(f'PDF sauvegardé à : {pdf_path}')
        print('(Installer poppler-utils pour afficher dans le notebook : sudo apt install poppler-utils)')
else:
    print('Aucune figure trouvée. Exécutez les cellules précédentes d\'abord.')

---
## Figure 5b — CIFAR-10 / Conv8 (Random Init → CIFAR-10)

**Temps estimé** : ~2-3h sur GPU  
**Dataset** : CIFAR-10 (téléchargement automatique)

In [ ]:
# Entraîner les modèles Conv8 (9 seeds, 60 epochs chacun)
run_train_until_complete('cifar10_conv8_sgd', gpu_id=0)

In [ ]:
# 4 variantes de transfert
for variant in ['transfer_init-cifar10_conv8',
                'transfer_init-cifar10_conv8_fast',
                'transfer_init-cifar10_conv8_noperm',
                'transfer_init-cifar10_conv8_usetarget']:
    run_cmd('transfer', variant, gpu_id=0)

In [ ]:
# Générer la figure
run_cmd('plot', 'section_4_1_cifar10_conv8', gpu_id=0)

---
## Figure 5f — CIFAR-10 → CIFAR-100 / Conv8 (Pretrained Init)

**Pré-requis** : `cifar10_conv8_sgd` entraîné (cellule précédente)

In [ ]:
# Fine-tuner sur CIFAR-100 split1 à partir des modèles CIFAR-10 pré-entraînés
run_train_until_complete('cifar10_to_cifar100-1_conv8_sgd', gpu_id=0)

In [ ]:
for variant in ['transfer_cifar10-cifar100_conv8',
                'transfer_cifar10-cifar100_conv8_fast',
                'transfer_cifar10-cifar100_conv8_noperm',
                'transfer_cifar10-cifar100_conv8_usetarget']:
    run_cmd('transfer', variant, gpu_id=0)

In [ ]:
run_cmd('plot', 'section_4_1_cifar10_cifar100_conv8', gpu_id=0)

---
## Section 3.3 — Ablations (Linear vs Actual, Uniform vs Cosine)

**Pré-requis** : `cifar10_conv8_sgd` entraîné

In [ ]:
# Ablation : scheduling uniforme vs cosinus
run_cmd('transfer', 'transfer_init-cifar10_conv8_ablation_scheduling_uniform', gpu_id=0)

In [ ]:
# Ablation : trajectoire linéaire vs fine-grained (checkpoints réels)
for variant in ['transfer_init-cifar10_conv8_ablation_linear',
                'transfer_init-cifar10_conv8_ablation_linear_fast',
                'transfer_init-cifar10_conv8_ablation_finegrained',
                'transfer_init-cifar10_conv8_ablation_finegrained_fast']:
    run_cmd('transfer', variant, gpu_id=0)

In [ ]:
# Générer les figures d'ablation
run_cmd('plot', 'section_3_3_uniform_vs_cosine', gpu_id=0)
run_cmd('plot', 'section_3_3_real_vs_linear', gpu_id=0)

---
## Figures 6a-b — Fine-tuning (Section 4.2)

Les résultats `ft_results.json` sont produits automatiquement lors du transfert. Il suffit de tracer.

In [ ]:
# Figure 6a : CIFAR-10 Conv8 fine-tuning
run_cmd('plot', 'section_4_2_cifar10_conv8', gpu_id=0)

# Figure 6b : CIFAR-10 → CIFAR-100 Conv8 fine-tuning  
run_cmd('plot', 'section_4_2_cifar10_cifar100_conv8', gpu_id=0)

---
## Figure 7 — Loss Landscape (Section 4.3)

Visualisation 1D (linear mode connectivity) et 2D (heatmap) de la loss landscape.

**Attention** : Ce script est GPU-intensif car il évalue des modèles interpolés en direct.

In [ ]:
run_cmd('plot', 'section_4_3_cifar10_conv8', gpu_id=0)

---
## Figures ImageNet (nécessitent ImageNet téléchargé)

### Pré-requis
- **ImageNet ILSVRC2012** (~150 GB) dans `__data__/imagenet/{train,val}/`
- **Stanford Cars** dans `__data__/`
- **CUB-200-2011** dans `__data__/`

### Temps de calcul
- Entraînement ResNet-18 × 6 seeds : **~3-7 jours** sur 1 GPU A100
- Fine-tuning Cars/CUB × 6 seeds : **~6-12h** après les modèles ImageNet

Décommenter les cellules ci-dessous si les datasets sont disponibles.

In [ ]:
# === DÉCOMMENTER SI IMAGENET EST DISPONIBLE ===

# # Figure 5c : ImageNet / ResNet-18
# run_train_until_complete('imagenet_resnet18_sgd', gpu_id=0)
# for v in ['transfer_init-imagenet_resnet18', 'transfer_init-imagenet_resnet18_fast',
#           'transfer_init-imagenet_resnet18_noperm', 'transfer_init-imagenet_resnet18_usetarget']:
#     run_cmd('transfer', v, gpu_id=0)
# run_cmd('plot', 'section_4_1_imagenet_resnet18', gpu_id=0)
# run_cmd('plot', 'section_4_2_imagenet_resnet18', gpu_id=0)

In [ ]:
# === DÉCOMMENTER SI IMAGENET + CARS SONT DISPONIBLES ===

# # Figure 5d : ImageNet → Cars
# run_train_until_complete('imagenet_to_cars_resnet18_sgd', gpu_id=0)
# for v in ['transfer_imagenet-cars_resnet18', 'transfer_imagenet-cars_resnet18_fast',
#           'transfer_imagenet-cars_resnet18_noperm', 'transfer_imagenet-cars_resnet18_usetarget']:
#     run_cmd('transfer', v, gpu_id=0)
# run_cmd('plot', 'section_4_1_imagenet_cars_resnet18', gpu_id=0)
# run_cmd('plot', 'section_4_2_imagenet_cars_resnet18', gpu_id=0)

In [ ]:
# === DÉCOMMENTER SI IMAGENET + CUB SONT DISPONIBLES ===

# # Figure 5e : ImageNet → CUB
# run_train_until_complete('imagenet_to_cub_resnet18_sgd', gpu_id=0)
# for v in ['transfer_imagenet-cub_resnet18', 'transfer_imagenet-cub_resnet18_fast',
#           'transfer_imagenet-cub_resnet18_noperm', 'transfer_imagenet-cub_resnet18_usetarget']:
#     run_cmd('transfer', v, gpu_id=0)
# run_cmd('plot', 'section_4_1_imagenet_cub_resnet18', gpu_id=0)
# run_cmd('plot', 'section_4_2_imagenet_cub_resnet18', gpu_id=0)

---
## Résumé des figures générées

Lister toutes les figures PDF produites dans `__outputs__/`.

In [ ]:
import glob

all_pdfs = sorted(glob.glob(str(REPO_ROOT / '__outputs__' / '**' / '*.pdf'), recursive=True))

if all_pdfs:
    print(f'✅ {len(all_pdfs)} figure(s) PDF générée(s) :\n')
    for pdf in all_pdfs:
        rel = os.path.relpath(pdf, REPO_ROOT)
        size_kb = os.path.getsize(pdf) / 1024
        print(f'  {rel}  ({size_kb:.1f} KB)')
else:
    print('❌ Aucune figure PDF trouvée. Exécutez les cellules de pipeline ci-dessus.')

print('\n--- Comparaison avec les figures originales ---')
original_pdfs = sorted(glob.glob(str(REPO_ROOT.parent / 'arXiv-2305.14122v2' / 'resources' / '*.pdf')))
print(f'{len(original_pdfs)} PDFs originaux dans arXiv-2305.14122v2/resources/')
for pdf in original_pdfs:
    print(f'  {os.path.basename(pdf)}')

---
## Correspondance Figures de l'article ↔ Commandes

| Figure | Section | Commande plot | Dataset nécessaire |
|--------|---------|---------------|-------------------|
| **Fig 1-2** | Intro | Diagrammes conceptuels (TikZ) | — |
| **Fig 3a-c** | 3.2 | PDFs pré-fournis dans `resources/` | — |
| **Fig 4** | 3.3 | `section_3_3_real_vs_linear` + `section_3_3_uniform_vs_cosine` | CIFAR-10 |
| **Fig 5a** | 4.1 | `section_4_1_mnist_mlp` | MNIST |
| **Fig 5b** | 4.1 | `section_4_1_cifar10_conv8` | CIFAR-10 |
| **Fig 5c** | 4.1 | `section_4_1_imagenet_resnet18` | ImageNet |
| **Fig 5d** | 4.1 | `section_4_1_imagenet_cars_resnet18` | ImageNet + Cars |
| **Fig 5e** | 4.1 | `section_4_1_imagenet_cub_resnet18` | ImageNet + CUB |
| **Fig 5f** | 4.1 | `section_4_1_cifar10_cifar100_conv8` | CIFAR-10 + CIFAR-100 |
| **Fig 6a** | 4.2 | `section_4_2_cifar10_conv8` | CIFAR-10 |
| **Fig 6b** | 4.2 | `section_4_2_cifar10_cifar100_conv8` | CIFAR-10/100 |
| **Fig 6c** | 4.2 | `section_4_2_imagenet_cars_resnet18` | ImageNet + Cars |
| **Fig 6d** | 4.2 | `section_4_2_imagenet_cub_resnet18` | ImageNet + CUB |
| **Fig 7** | 4.3 | `section_4_3_cifar10_conv8` | CIFAR-10 |
| **Fig 8** | 4.3 | Pas de script dédié | ImageNet + Cars/CUB |